## Dataset cachexia - Exploração de dados, pré-processamento e análise estatística univariada
Nesta secção vamos trabalhar com os dados do dataset Cachexia : 

*"Concentrations of 63 metabolites in 77 urine samples from cancer patients, measured by 1H NMR (Eisner et al, 2011). Metadata defines the two groups: control and cachexic patients."*

Importação e análise da dimensão dos dados

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
# para desenhar os gráficos dentro do notebook
%matplotlib inline


In [ ]:
data = pd.read_csv('data_cachexia.csv',sep=',', index_col = 0)
meta = pd.read_csv('meta_cachexia.csv',sep=',', index_col = 0)
print (data.shape)
print (meta.shape)
data.head()

Sumário dos dados

In [ ]:
data.describe()
#data.columns

Sumário dos metadados 

In [ ]:
meta.describe()


Frequência de cada categoria nos metadados

In [ ]:
meta.groupby("Muscle.loss").size()

Verificar que não há valores em falta.

In [ ]:
data.isnull().sum().sum()

Para cada coluna apresentar o intervalo de valores

In [ ]:
data.apply(lambda x: str(x.min())+"---"+str( x.max()))

Criar um gráfico que permita visualizar os valores dos metabolitos "Alanine", "Citrate" e "Creatinine" para os pacientes de control. 

In [ ]:
data_control = data[meta["Muscle.loss"]=="control"]
data_control.iloc[:,[13,17,19]].plot.bar()

Exemplo de discretização de dados - alterar os dados do metabolito Citrate de valor continuo para discretos: 
    "elevado", "baixo" consoante o valor seja: maior ou menor que a média.

In [ ]:
cit = data.Citrate.copy()

mean = cit.mean()
cit[cit>mean]="elevado"
cit[cit!="elevado"]="baixo"
cit

Standardização dos dados

In [ ]:
from sklearn import preprocessing
array_data = preprocessing.scale(data)#, with_mean = False)
data_scale =  pd.DataFrame(array_data, index =data.index , columns = data.columns)
data_control = data_scale[meta["Muscle.loss"]=="control"]
data_control.iloc[:,[13,17,19]].plot.bar()

Cria um boxplot para ver a distribuição dos valores dos metabolitos Asparagine, Acetate, Succinate.

In [ ]:
indexes = [ind for ind in range(len(data.columns)) if data.columns[ind] in ['Asparagine', 'Acetate', 'Succinate']]
sel_data = data.iloc[:,indexes].values
labels = data.columns[indexes]

#ou 
sel_data = data.loc[:,['Asparagine', 'Acetate', 'Succinate']].values
labels = ['Asparagine', 'Acetate', 'Succinate']

plt.boxplot(sel_data)
plt.xticks([1,2,3],labels)
plt.title(" Boxplot")
plt.show()


In [ ]:
indexes = [ind for ind in range(len(data.columns)) if data.columns[ind] in ['Sucrose', 'Glucose', 'Xylose']]
sel_data = data.iloc[:,indexes].values
labels = data.columns[indexes]
plt.boxplot(sel_data)
plt.xticks([1,2,3],labels)
plt.title(" Boxplot")
plt.show()

Transformação logarítmica

In [ ]:
log_data = np.log2(data)
log_data.apply(lambda x: str(x.min())+"---"+str( x.max()))

In [ ]:
sel_log_data = log_data.iloc[:,indexes].values

plt.boxplot(sel_log_data)
plt.xticks([1,2,3],labels)
plt.title(" Boxplot")
plt.show()

### Exemplos com testes estatísticos
Verifica se a distribuição do metabolito Alanine segue a distribuição normal.

In [ ]:
import statsmodels.api as sm
from scipy import stats

In [ ]:
plt.close()
data_alanine = data.Alanine
plt.hist(data_alanine)

In [ ]:
plt.close()
from sklearn import preprocessing
sm.qqplot(preprocessing.scale(data_alanine), line='45')


In [ ]:
w, p_value = stats.shapiro(data_alanine)
p_value
# rejeita HO ... logo dados não seguem a distribuição normal


A concentração de Alanina entre as duas categorias é semelhante?

In [ ]:
data_alanine = data.Alanine
control = data_alanine[meta["Muscle.loss"]=="control"]
cachexic = data_alanine[meta["Muscle.loss"]=="cachexic"]

plt.hist(control)
plt.show()
plt.hist(cachexic)
plt.show()


In [ ]:
stats.ttest_ind(control,cachexic)
# Rejeita-se a hipose nula (as duas amostras não tem média igual)

In [ ]:
plt.close()
plt.boxplot([control,cachexic])
plt.show()

Quais os metabolitos que têm concentrações significativamente diferentes entre os dois grupos: control e cachexic. 

In [ ]:
type(data[meta['Muscle.loss']=='control'])

In [ ]:
metabs_de_tt = lambda x : stats.ttest_ind(x[meta['Muscle.loss']=='control'],x[meta['Muscle.loss']=='cachexic'])

data.columns[data.apply(lambda x: metabs_de_tt(x)[1]<0.05)]

Testes não paramétricos - Mann-Whitney U test

In [ ]:
stats.mannwhitneyu(control, cachexic)

In [ ]:
metabs_de_mw = lambda x : stats.mannwhitneyu(x[meta['Muscle.loss']=='control'],x[meta['Muscle.loss']=='cachexic'])

data.columns[data.apply(lambda x: metabs_de_mw(x)[1]<0.05)]

Correções para testes múltiplos

In [ ]:
pvalues_tt = data.apply(lambda x: metabs_de_tt(x)[1])
pvalues_tt
type(pvalues_tt)

In [ ]:
import statsmodels.stats as sm
corr_pvalues = sm.multitest.multipletests(pvalues_tt, method = "fdr_bh")[1]
corr_pvalues

### Correlações

In [ ]:
data_glucose = data.Glucose
data_sucrose = data.Sucrose

plt.scatter(data_glucose, data_sucrose)
plt.show()

In [ ]:
corr, _ = stats.pearsonr(data_glucose, data_sucrose)
corr

In [ ]:
sp_corr, _ = stats.spearmanr(data_glucose, data_sucrose)
sp_corr

In [ ]:
corr_matrix = data.corr()
corr_matrix

In [ ]:
type(corr_matrix)

In [ ]:
corr_matrix_sp = data.corr(method = "spearman")
corr_matrix_sp

In [ ]:
import seaborn as sn
sn.heatmap(corr_matrix.iloc[1:10,1:10], annot=True)
plt.show()

In [ ]:
sn.heatmap(corr_matrix.iloc[1:20,1:20], annot=False)
plt.show()

Análise de variância / regressão linear

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

lm = ols("data.Leucine~data.Valine", data).fit()
anova_lm(lm)


In [ ]:
lm.summary()

In [ ]:
lm.rsquared

In [ ]:
lm2 = ols("data.Leucine~data.Valine+ data.Glucose", data).fit()

In [ ]:
lm2.summary()

In [ ]:
data["MuscleLoss"] = meta["Muscle.loss"]

In [ ]:
lm3 = ols("data.Leucine~data.MuscleLoss", data).fit()

In [ ]:
lm3.summary()